In [100]:
!pip install tensorflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 MB 7.4 MB/s eta 0:00:0000:0100:01
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached ast

In [106]:
from tensorflow.keras.tuner import Hyperband  # Use Keras Tuner from TensorFlow
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow.keras.tuner'

In [10]:
df = pd.read_csv('Dataset_with_new_features.csv')


In [17]:
# Check the total number of NaNs in each column
nan_summary = df.isna().sum()
print("\nCount of NaN values per column:")
print(nan_summary[nan_summary > 0])  # Show only columns with NaN values


Count of NaN values per column:
source    911
dtype: int64


In [16]:
df = df[~df['perplexity'].isna()]


In [18]:
# Identify rows where 'generated' == 'generated'
rows_to_drop = df[df['generated'] == 'generated'].index

# Drop these rows
dataset = df.drop(rows_to_drop)

In [19]:
dataset.shape

(9610, 29)

In [93]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create a copy of the dataset
dataset_em = dataset.copy()

# Generate embeddings for all texts at once
embeddings = model.encode(dataset['text'].tolist(), show_progress_bar=True).tolist()



Batches: 100%|██████████| 301/301 [03:30<00:00,  1.43it/s]


In [98]:
print(embeddings[0])

[-0.016862688586115837, -0.01975325122475624, 0.00672335596755147, -0.046849966049194336, -0.03179958835244179, 0.04981544613838196, 0.05810887739062309, 0.0739719569683075, -0.026174422353506088, -0.016984445974230766, 0.05613713711500168, 0.14367012679576874, 0.08398312330245972, 0.002008205745369196, -0.038107581436634064, -0.005695142317563295, 0.04792719706892967, -0.07158791273832321, -0.05759040266275406, 0.10773701220750809, -0.08626166731119156, -0.03069588914513588, -0.012663532048463821, 0.06383758038282394, -0.13041767477989197, 0.0018171073170378804, -0.04379447177052498, -0.044647444039583206, 0.04152967408299446, -0.0277634859085083, 0.061313025653362274, 0.045239586383104324, 0.05202922970056534, 0.04830915108323097, -0.0033080652356147766, -0.06978365778923035, 0.05413535609841347, 0.02697584219276905, -0.017152605578303337, -0.06612619757652283, 0.021504029631614685, -0.10931337624788284, -0.0981716439127922, 0.001287044957280159, 0.033099621534347534, -0.062804564833

In [95]:
# Add embeddings as a new column
dataset_em['embeddings'] = np.array(embeddings.tolist())

# Print the new embeddings column
print(dataset_em[['text', 'embeddings']].head())


AttributeError: 'list' object has no attribute 'tolist'

In [73]:
features = dataset_em.drop(columns=['essay_id', 'text', 'generated', 'source', 'temp', 'word_count', 'average_word_length',"flesch_kincaid", "punctuation_count", 'noun_count', 'verb_count', 'adjective_count', 'adverb_count', 'pos_counts', 'pos_density', 'misspelled_word_count', 'ai_ratio'  ])


In [74]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9610 entries, 0 to 9610
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   vocabulary_richness  9610 non-null   float64
 1   gunning_fog          9610 non-null   float64
 2   smog_index           9610 non-null   float64
 3   polarity             9610 non-null   float64
 4   subjectivity         9610 non-null   float64
 5   sentence_length      9610 non-null   float64
 6   punctuation_density  9610 non-null   float64
 7   noun_density         9610 non-null   float64
 8   verb_density         9610 non-null   float64
 9   adjective_density    9610 non-null   float64
 10  adverb_density       9610 non-null   float64
 11  perplexity           9610 non-null   float64
 12  embeddings           9610 non-null   object 
dtypes: float64(12), object(1)
memory usage: 1.3+ MB


In [76]:
X = features
y = dataset['generated']  # Target column
numerical_cols = features.columns
print("Numerical Cols: ", numerical_cols)
numerical_cols = numerical_cols.drop('embeddings')

Numerical Cols:  Index(['vocabulary_richness', 'gunning_fog', 'smog_index', 'polarity',
       'subjectivity', 'sentence_length', 'punctuation_density',
       'noun_density', 'verb_density', 'adjective_density', 'adverb_density',
       'perplexity', 'embeddings'],
      dtype='object')


In [77]:

scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])


In [87]:
print(X.columns)
print(len(dataset_em['embeddings'].iloc[0]))
print(len(dataset_em['embeddings']))

Index(['vocabulary_richness', 'gunning_fog', 'smog_index', 'polarity',
       'subjectivity', 'sentence_length', 'punctuation_density',
       'noun_density', 'verb_density', 'adjective_density', 'adverb_density',
       'perplexity', 'embeddings'],
      dtype='object')
384
9610


In [79]:
print(f"Length of X: {len(X)}")
print(f"Length of y: {len(y)}")

Length of X: 9610
Length of y: 9610


In [53]:
common_indices = X.index.intersection(y.index)
X = X.loc[common_indices]
y = y.loc[common_indices]

In [80]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 70% train, 30% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into 50% validation and 50% test

print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 5766
Validation set size: 1922
Test set size: 1922


In [81]:
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    max_iter=100,
    random_state=42,
    verbose=True
)

In [82]:
mlp.fit(X_train, y_train)


ValueError: setting an array element with a sequence.

In [58]:
y_val = y_val.astype(int)
print(y_val)

2830    1
5134    0
6851    0
1658    1
4892    1
       ..
5961    0
1672    1
1383    1
1231    0
1851    0
Name: generated, Length: 1922, dtype: int64


In [59]:
# Step 3: Evaluate on Validation Set
y_val_pred = mlp.predict(X_val)
y_val_prob = mlp.predict_proba(X_val)[:, 1]  # Probabilities for class 1



In [60]:
print(y_val_prob[:10])
print(y_val_pred)
y_val_pred = y_val_pred.astype(int)

[3.06633718e-01 1.12875244e-07 8.08544919e-01 9.99956589e-01
 1.00000000e+00 1.14590852e-04 1.00000000e+00 1.00000000e+00
 3.17639264e-22 2.26404617e-08]
['0' '0' '1' ... '0' '0' '0']


In [61]:

val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred, average='binary')
val_recall = recall_score(y_val, y_val_pred, average='binary')
val_roc_auc = roc_auc_score(y_val, y_val_prob)

print("\nValidation Metrics:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")
print(f"ROC AUC Score: {val_roc_auc:.4f}")


Validation Metrics:
Accuracy: 0.9506
Precision: 0.9543
Recall: 0.9301
ROC AUC Score: 0.9842
